In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

In [2]:
# 📌 분석 대상 가게 리스트 (모바일 페이지 URL)
stores = [
    {'store_id': 'A', 'name': '잔잔 범계역점', 'url': 'https://m.place.naver.com/restaurant/1503985284/review/visitor'},
    {'store_id': 'B', 'name': '이자카야 이태원', 'url': 'https://m.place.naver.com/restaurant/36538366/review/visitor'},
    {'store_id': 'C', 'name': '파랑새야', 'url': 'https://m.place.naver.com/restaurant/1595065259/review/visitor'}
]

all_reviews = []

In [3]:
for store in stores:
    # 💡 각 가게마다 독립적인 드라이버(새 창)를 생성하여 안정성을 확보합니다.
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)
    
    print(f"\n--- {store['name']} 리뷰 수집 시작 ---")
    
    try:
        driver.get(store['url'])
        time.sleep(2)
        
        # '리뷰' 탭이 클릭 가능할 때까지 기다린 후 클릭합니다.
        review_tab_selector = "span.veBoZ"
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, review_tab_selector))).click()
        time.sleep(1)

        # '펼쳐서 더보기' 버튼의 XPath (텍스트 기반으로 찾아 안정적)
        more_button_xpath = "//span[contains(text(), '펼쳐서 더보기')]"
        review_elements_selector = "div.pui__vn15t2 > a:first-child"
        
        # 현재 로드된 리뷰 개수가 20개가 될 때까지 '더보기' 버튼을 클릭합니다.
        while len(driver.find_elements(By.CSS_SELECTOR, review_elements_selector)) < 40:
            try:
                more_btn = wait.until(EC.presence_of_element_located((By.XPATH, more_button_xpath)))
                driver.execute_script("arguments[0].click();", more_btn)
                time.sleep(1)
            except Exception:
                # 더 이상 '더보기' 버튼이 없으면 루프를 종료합니다.
                break

        # 💡 실제 리뷰 텍스트를 담고 있는 첫 번째 자식 a 태그를 선택합니다.
        review_elements = driver.find_elements(By.CSS_SELECTOR, review_elements_selector)

        count = 0
        for elem in review_elements:
            if count >= 40:
                break
            text = elem.text.strip()
            if text:
                all_reviews.append({
                    'store_id': store['store_id'],
                    'store_name': store['name'],
                    'review': text
                })
                count += 1
        
        print(f"✅ {store['name']}에서 {count}개 리뷰 수집 완료")

    except Exception as e:
        print(f"❌ {store['name']} 리뷰 수집 중 오류 발생: {type(e).__name__} - {e}")
        
    finally:
        # 작업이 성공하든 실패하든 현재 드라이버(창)는 확실히 종료합니다.
        driver.quit()

print("\n--- 모든 가게 리뷰 수집 완료 ---")

# 수집된 모든 리뷰를 CSV 파일로 저장합니다.
if all_reviews:
    df = pd.DataFrame(all_reviews)
    if not os.path.exists("../data"):
        os.makedirs("../data")
    df.to_csv("../data/raw_reviews.csv", index=False, encoding="utf-8-sig")
    print(f"\n총 {len(df)}개 리뷰를 './data/raw_reviews.csv' 파일로 저장했습니다.")
    print("----- 데이터 샘플 -----")
    print(df.head())
else:
    print("\n❌ 수집된 리뷰가 없습니다.")


--- 잔잔 범계역점 리뷰 수집 시작 ---
✅ 잔잔 범계역점에서 40개 리뷰 수집 완료

--- 이자카야 이태원 리뷰 수집 시작 ---
✅ 이자카야 이태원에서 37개 리뷰 수집 완료

--- 파랑새야 리뷰 수집 시작 ---
✅ 파랑새야에서 40개 리뷰 수집 완료

--- 모든 가게 리뷰 수집 완료 ---

총 117개 리뷰를 './data/raw_reviews.csv' 파일로 저장했습니다.
----- 데이터 샘플 -----
  store_id store_name                                             review
0        A    잔잔 범계역점  언제나 애정하는 잔잔입니다:)\n오늘은\n다양하게 먹어봤어오! 모찌리도후는\n먹고싶...
1        A    잔잔 범계역점  인기많은 모찌리도후 오늘은 마지막으로 겟 했습니다:)\n해물야끼우동 처음 먹어보는데...
2        A    잔잔 범계역점  항상 인계동에서만 가봤다가 범계는 처음 가봤어요. 공휴일 저녁이라 역시 웨이팅이 있...
3        A    잔잔 범계역점  #범계술집#범계맛집#범계모임장소\n남편과 데이트 할때면 무조건 오는 잔잔~^^\n맛...
4        A    잔잔 범계역점  범계술집하면 믿고 먹는 잔잔이죠 !! 안주 뭘 시켜도 너무 맛있고 하이볼 음료 삼아...
